<a href="https://colab.research.google.com/github/tobiasuruali/WebScraper_PolAds/blob/main/code/test/test_pol_ads_transcribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip -qqq install ipython-autotime
%load_ext autotime

time: 1.1 ms (started: 2023-03-31 09:00:39 +00:00)


In [ ]:
pip -qqq install git+https://github.com/openai/whisper.git 

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 KB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
time: 31.8 s (started: 2023-03-31 08:53:08 +00:00)


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab_Data')
print(os.listdir())
print(os.getcwd())

['test_pol_ad.mp3', 'test_pol_ad_2.mp3']
/content/drive/MyDrive/Colab_Data


# Download Video Audio

Download the video audio with the help of pytube. This will then be used to transcribe the data.


In [ ]:
# Install the library
!pip -qqq install pytube

# import the module
from pytube import YouTube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 KB 4.7 MB/s eta 0:00:00
time: 5.42 s (started: 2023-03-31 08:53:39 +00:00)


In [ ]:
def video_to_audio(video_URL, destination, final_filename):

  # Get the video
  video = YouTube(video_URL)

  # Convert video to Audio
  audio = video.streams.filter(only_audio=True).first()

  # Save to destination
  output = audio.download(output_path = destination)

  _, ext = os.path.splitext(output)
  new_filename = final_filename + '.mp3'

  # Change the name of the file
  os.rename(output, new_filename)
  return new_filename

# Video to audio
video_URL = 'https://www.youtube.com/watch?v=WHQ6kOdVIGE&embeds_euri=https%3A%2F%2Fadstransparency.google.com%2F&feature=emb_imp_woyt'
destination = "."
final_filename = "test_pol_ad_2"
new_filename = video_to_audio(video_URL, destination, final_filename)

time: 1.44 s (started: 2023-03-31 08:53:45 +00:00)


# OpenAI Whisper Trial 

Using OpenAI's Whisper model. Using the large model to try out transcribing and evaluate performance.

[Open AI Whisper](https://github.com/openai/whisper)


In [ ]:

# Import the libraries 
import whisper
import torch 
import os

# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


# Load the model 
whisper_model = whisper.load_model("large", device=device)

cpu


100%|█████████████████████████████████████| 2.87G/2.87G [00:31<00:00, 99.2MiB/s]


time: 1min 25s (started: 2023-03-31 08:53:46 +00:00)


In [ ]:
print(new_filename)

test_pol_ad_2.mp3
time: 722 µs (started: 2023-03-31 08:55:12 +00:00)


In [ ]:
audio_file = new_filename
result = whisper_model.transcribe(audio_file)

     

/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


time: 2min 16s (started: 2023-03-31 08:55:12 +00:00)


In [ ]:
print(result["text"])


 When Paul Young says he supports outlawing abortion with no exceptions for rape or incest, believe him. And when he says that he believes abortion is a made-up right, believe him. Say no to Paul Young.
time: 635 µs (started: 2023-03-31 08:57:29 +00:00)


In [ ]:
print(result)

{'text': ' When Paul Young says he supports outlawing abortion with no exceptions for rape or incest, believe him. And when he says that he believes abortion is a made-up right, believe him. Say no to Paul Young.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.0, 'text': ' When Paul Young says he supports outlawing abortion with no exceptions for rape or incest, believe him.', 'tokens': [50364, 1133, 4552, 8160, 1619, 415, 9346, 484, 5901, 278, 22902, 365, 572, 22847, 337, 22846, 420, 834, 377, 11, 1697, 796, 13, 50714], 'temperature': 0.0, 'avg_logprob': -0.23711765937085422, 'compression_ratio': 1.467153284671533, 'no_speech_prob': 0.4062627851963043}, {'id': 1, 'seek': 0, 'start': 7.0, 'end': 12.0, 'text': ' And when he says that he believes abortion is a made-up right, believe him.', 'tokens': [50714, 400, 562, 415, 1619, 300, 415, 12307, 22902, 307, 257, 1027, 12, 1010, 558, 11, 1697, 796, 13, 50964], 'temperature': 0.0, 'avg_logprob': -0.23711765937085422, 'compression

# Faster Whisper Trial

Attempt at using the faster-whisper package for transcribing

[Faster Whisper](https://github.com/guillaumekln/faster-whisper)

In [ ]:
pip -qqq install faster-whisper

In [ ]:
from faster_whisper import WhisperModel

model_size = "large-v2"

# # Run on GPU with FP16
# model = WhisperModel(model_size, device="cuda", compute_type="float16")

# # or run on GPU with INT8
# # model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
model = WhisperModel(model_size, device="cpu", compute_type="int8")



Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# segments, info = model.transcribe(new_filename, beam_size=5)

# print("Detected language '%s' with probability %f" % (info.language, info.language_probability))



# # for segment in segments:
# #     print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

In [ ]:
filename_mp3 = 'test_pol_ad_2.mp3'

def transcribe_and_save(new_filename):
    segments, info = model.transcribe(new_filename, beam_size=5)
    output = ""
    for segment in segments:
        output += segment.text
    return output, new_filename

output, filename_mp3 = transcribe_and_save(filename_mp3)
print(output)

 When Paul Young says he supports outlawing abortion with no exceptions for rape or incest, believe him. And when he says that he believes abortion is a made-up right, believe him. Say no to Paul Young.
time: 1min 22s (started: 2023-03-31 09:00:46 +00:00)


In [ ]:
import pandas as pd

df = pd.DataFrame({'filename': [filename_mp3], 'output': [output]})
print(df)

            filename                                             output
0  test_pol_ad_2.mp3   When Paul Young says he supports outlawing ab...
time: 951 ms (started: 2023-03-31 09:03:27 +00:00)
